In [3]:
import cv2
import numpy as np
import glob
import os

# Path to the directory containing the images
images_dir = 'C:/Users/mfaiz/Desktop/PROJECT/Sorting of Batteries/Camera Caliberation/Images'
images_path = os.path.join(images_dir, '*.jpg')

# Set the parameters for the calibration pattern
pattern_size = (8, 11)  # Number of inner corners per a chessboard row and column
square_size = 1.0  # Size of a square in your defined unit (here, it will be in the same unit as your real-world object)

# Prepare object points
objp = np.zeros((np.prod(pattern_size), 3), np.float32)
objp[:, :2] = np.indices(pattern_size).T.reshape(-1, 2)
objp *= square_size

# Arrays to store object points and image points from all the images
objpoints = []  # 3d point in real world space
imgpoints = []  # 2d points in image plane

# Process each image in the directory
for image_file in glob.glob(images_path):
    img = cv2.imread(image_file)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detect the symmetric circle grid in the image
    ret, corners = cv2.findCirclesGrid(gray, pattern_size, cv2.CALIB_CB_SYMMETRIC_GRID)

    if ret:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw the corners on the original image
        img_with_corners = cv2.drawChessboardCorners(img.copy(), pattern_size, corners, ret)

        # Display the image with corners
        cv2.namedWindow('Corners', cv2.WINDOW_NORMAL)  # Create a resizable window
        cv2.imshow('Corners', img_with_corners)
        cv2.waitKey(500)  # Display for 500 milliseconds

cv2.destroyAllWindows()

# Check if any pattern was detected
if objpoints and imgpoints:
    # Perform camera calibration
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
    
    # Print calibration results
    print("Camera matrix (intrinsic parameters):\n", mtx)
    print("Distortion coefficients:\n", dist)
    
    # Save calibration parameters
    np.savez('calibration_data.npz', mtx=mtx, dist=dist)
    print("Calibration data saved to calibration_data.npz")
    
    print("Camera calibration and saving parameters completed.")
else:
    print("No patterns were detected in the provided images.")


Camera matrix (intrinsic parameters):
 [[4.82935649e+03 0.00000000e+00 2.24440232e+03]
 [0.00000000e+00 5.94853839e+03 1.15263931e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion coefficients:
 [[ 1.39123270e+00 -8.97894422e+01 -3.51855167e-02 -6.96907485e-03
   1.20292362e+03]]
Calibration data saved to calibration_data.npz
Camera calibration and saving parameters completed.
